# Поиск закономерностей в данных

In [3]:
import pandas as pd
import numpy as np
import scipy

## Correlation. Practice

### Question 1

Есть ли связь между неграмотностью и рождаемостью? Для 94 стран, уровень неграмотности женщин в которых больше 5%, известны доля неграмотных среди женщин старше 15 (на 2003 год) и средняя рождаемость на одну женщину (на 2005 год).

Чему равен выборочный коэффициент корреляции Пирсона между этими двумя признаками? Округлите до четырёх знаков после десятичной точки.

In [2]:
illiteracy = pd.read_csv('illiteracy.txt', sep = '\t', index_col = 'Country')
illiteracy.head()

IOError: File illiteracy.txt does not exist

In [ ]:
def corr_answer(df, method = 'pearson'):
    return round(float(df.corr(method = method).iloc[0,1]), 4)

In [ ]:
print('Pearson Correlation =', corr_answer(illiteracy))

### Question 2

Чему равен выборочный коэффициент корреляции Спирмена признаков из предыдущего вопроса? Округлите до четырёх знаков после десятичной точки.

In [ ]:
print('Spearman Correlation =', corr_answer(illiteracy, method = 'spearman'))

## Correlation. Quiz

### Question 1

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [ ]:
water = pd.read_csv('water.txt', sep = '\t')
water.head()

In [ ]:
print('Pearson Correlation =', corr_answer(water[['mortality', 'hardness']]))

### Question 2

В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.

In [ ]:
print('Spearman Correlation =', corr_answer(water[['mortality', 'hardness']], method = 'spearman'))

### Question 3

Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [ ]:
north_cor = np.abs(corr_answer(water[(water['location'] == 'North')][['mortality', 'hardness']]))
south_cor = np.abs(corr_answer(water[(water['location'] == 'South')][['mortality', 'hardness']]))

print('ABS Minimum Perason Correlation =', round(float(np.minimum(north_cor, south_cor)), 4))

### Question 4

Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

| 0 | 1  
------------- | -------------|
0  | a | b 
1  | c | d 

$$\text{Формула корреляции Мэтьюса:}$$
  
$$MCC_{X_1, X_2} =  \frac{ad-bc}{\sqrt{(a+b)(a+c)(b+d)(c+d)}}$$  

| 0 | 1  
------------- | -------------|
0  | 203 | 239 
1  | 718 | 515 

In [ ]:
mcor = (203*515 - 718*239)/np.sqrt((203+239)*(203+718)*(515+239)*(515+718))

print('Matthew Correlation =', round(mcor,3))

### Question 5

В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры.

In [ ]:
from scipy.stats import chi2_contingency

bar = np.array([[203, 239], [718, 515]])

print('p-value =', chi2_contingency(bar)[1])

### Question 6

В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [ ]:
fem = np.append(np.ones(203), np.zeros(718))
mal = np.append(np.ones(239), np.zeros(515))

In [ ]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [ ]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(mal, fem))

print('Answer is', round(proportions_diff_confint_ind(mal, fem)[0], 4))

### Question 7

Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры.

In [ ]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [ ]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [ ]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(mal, fem)))
print('Answer is', 6)

### Question 8

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?" Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

|Не доволен|Более или менее|Доволен
---|---|---
Не очень счастлив|197|111|33
Достаточно счастлив|382|685|331
Очень счастлив|110|	342|333


In [ ]:
happy = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

print('Chi2 Stats =', round(float(chi2_contingency(happy)[0]), 4))

### Question 9

На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры.

In [ ]:
print('p-value =', chi2_contingency(happy)[1])

### Question 10

Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

$$\phi_c(X^{n}_1, X^{n}_2) = \sqrt{\frac{ \chi^2(X^{n}_1, X^{n}_2)}{n(min(K_1, K_2) - 1)}}$$

In [ ]:
cramer = np.sqrt(chi2_contingency(happy)[0]/(np.sum(happy) * 2))

print('Cramer Correlation =', round(cramer, 4))

## Множественная проверка гипотез. Quiz 

### Question 2

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. 

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [ ]:
aucs = pd.read_csv('AUCs.txt', sep = '\t', index_col = 0)

aucs.head()

In [ ]:
from scipy import stats

In [ ]:
%%time 
wilcox_data = []

for i, lhs_column in enumerate(aucs.columns):
    for j, rhs_column in enumerate(aucs.columns):
        if i >= j:
            continue
        
        stat, p = stats.wilcoxon(aucs[lhs_column], aucs[rhs_column])
        wilcox_data.append([lhs_column, rhs_column, stat, p])

In [ ]:
#C4.5, C4.5+m
wilcox_data

### Question 3

Сколько статистически значимых на уровне 0.05 различий мы обнаружили? **4**

### Question 4

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества? **m**

### Question 5

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом? **0**

In [ ]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [ ]:
models_wilcox = pd.DataFrame.from_records(wilcox_data)
models_wilcox.columns = ['model_A', 'model_B', 'wilcox_stats', 'p']

models_wilcox.head()

In [ ]:
reject, p_corrected, a1, a2 = multipletests(models_wilcox.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [ ]:
models_wilcox['p_corrected'] = p_corrected
models_wilcox['reject'] = reject

In [ ]:
models_wilcox.reject.value_counts()

### Question 6

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга? **3**

In [ ]:
reject, p_corrected, a1, a2 = multipletests(models_wilcox.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [ ]:
models_wilcox['p_corrected'] = p_corrected
models_wilcox['reject'] = reject

In [ ]:
models_wilcox.reject.value_counts()

## Практика построения регрессии. Quiz

### Question 1

Давайте проанализируем данные опроса 4361 женщин из Ботсваны:

О каждой из них мы знаем:

* сколько детей она родила (признак ceb)
* возраст (age)
* длительность получения образования (educ)
* религиозная принадлежность (religion)
* идеальное, по её мнению, количество детей в семье (idlnchld)
* была ли она когда-нибудь замужем (evermarr)
* возраст первого замужества (agefm)
* длительность получения образования мужем (heduc)
* знает ли она о методах контрацепции (knowmeth)
* использует ли она методы контрацепции (usemeth)
* живёт ли она в городе (urban)
* есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [4]:
botswana = pd.read_csv('botswana.tsv', sep = '\t')

botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [5]:
#4
botswana.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

### Question 2

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [6]:
#1834
print('Shape: {}\nShape without NA: {}\nDifference = {}'.format(
    botswana.shape, botswana.dropna().shape, botswana.shape[0]-botswana.dropna().shape[0]))

Shape: (4361, 15)
Shape without NA: (1834, 15)
Difference = 2527


### Question 3

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

1. Создайте признак nevermarr, равный единице там, где в agefm пропуски.
2. Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице $X$ будет мультиколлинеарность.
3. Замените NaN в признаке agefm на $c_{agefm}=0$
4. У объектов, где nevermarr = 1, замените NaN в признаке heduc на $c_{heduc_1}=-1$ (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке heduc?

In [7]:
botswana['nevermarr'] = np.where(botswana['agefm'].isnull(), 1, 0)

In [8]:
botswana.drop(['evermarr'], axis = 1, inplace=True)

In [9]:
botswana['agefm'] = np.where(botswana['agefm'].isnull(), 0, botswana['agefm'])

In [10]:
botswana['heduc'] = np.where((botswana['heduc'].isnull()) & (botswana['nevermarr'] == 1), -1, botswana['heduc'])

In [11]:
botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [12]:
print('heduc NaNs left =', np.sum(botswana.heduc.isnull()))

('heduc NaNs left =', 123)


### Question 4

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения ($c_{idlnchld}=-1$, $c_{heduc_2}=-2$ (значение -1 мы уже использовали), $c_{usemeth}=-1$).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [13]:
botswana['idlnchld_noans'] = np.where(botswana['idlnchld'].isnull(), 1, 0)
botswana['heduc_noans'] = np.where(botswana['heduc'].isnull(), 1, 0)
botswana['usemeth_noans'] = np.where(botswana['usemeth'].isnull(), 1, 0)

In [14]:
botswana['idlnchld'] = np.where(botswana['idlnchld'].isnull(), -1, botswana['idlnchld'])
botswana['heduc'] = np.where(botswana['heduc'].isnull(), -2, botswana['heduc'])
botswana['usemeth'] = np.where(botswana['usemeth'].isnull(), -1, botswana['usemeth'])

In [15]:
botswana.dropna(inplace = True)

In [16]:
print('New Shape =', botswana.shape[0]*botswana.shape[1])

('New Shape =', 78264)


### Question 5

Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации $R^2$? Округлите до трёх знаков после десятичной точки.

In [17]:
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

In [18]:
botswana.columns

Index([u'ceb', u'age', u'educ', u'religion', u'idlnchld', u'knowmeth',
       u'usemeth', u'agefm', u'heduc', u'urban', u'electric', u'radio', u'tv',
       u'bicycle', u'nevermarr', u'idlnchld_noans', u'heduc_noans',
       u'usemeth_noans'],
      dtype='object')

In [19]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data = botswana)
fitted = m1.fit()
print(fitted.summary())

#0.644

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Tue, 08 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:24:33   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

### Question 7

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [20]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [21]:
m2 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data = botswana)
fitted = m2.fit(cov_type='HC1')
fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Tue, 08 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:24:33   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0263      0.266     -3.863      0.000      -1.547      -0.506
religion[T.other]         -0.0830      0.078     -1.067      0.286      -0.235       0.069
religion[T.protestant]    -0.0149      0.078     -0.192      0.848      -0.167       0.137
religion[T.spirit]        -0.0191      0.071     -0.268      0.789      -0.159       0.121
age                        0.1703      0.004     38.627      0.000       0.162       0.179
educ                      -0.0724      0.007     -9.924      0.000      -0.087      -0.058
idlnchld                   0.0760      0.015      5.236      0.000       0.048       0.104
knowmeth                   0.5564      0.174      3.190      0.001       0.215       0.898
usemeth                    0.6473      0.052     12.478      0.000       0.546       0.749
agefm                     -0.0604      0.010     -6.174      0.000      -0.080      -0.041
heduc                     -0.0551      0.009     -6.126      0.000      -0.073      -0.037
urban                     -0.2137      0.046     -4.667      0.000      -0.303      -0.124
electric                  -0.2685      0.072     -3.732      0.000      -0.410      -0.128
radio                     -0.0235      0.053     -0.446      0.656      -0.127       0.080
tv                        -0.1451      0.082     -1.766      0.077      -0.306       0.016
bicycle                    0.2139      0.048      4.412      0.000       0.119       0.309
nevermarr                 -2.2393      0.202    -11.082      0.000      -2.635      -1.843
idlnchld_noans             0.6539      0.216      3.029      0.002       0.231       1.077
heduc_noans               -0.8724      0.191     -4.556      0.000      -1.248      -0.497
usemeth_noans              0.7652      0.213      3.590      0.000       0.347       1.183
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         361.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

### Question 8

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [22]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data = botswana)
fitted = m3.fit()

In [23]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [24]:
m4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data = botswana)
fitted = m4.fit(cov_type='HC1')

In [25]:
#0.4672 Модель не стала хуже
print("F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m4.fit()))

F=0.919236, p=0.467231, k1=5.000000


### Question 9

Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением $c_{usemeth }=-1$, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости.

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [26]:
m5 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans', data = botswana)
fitted = m5.fit()

In [27]:
print("F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m4.fit()))

F=0.919236, p=0.467231, k1=5.000000


In [28]:
#40
m4.fit().compare_f_test(m5.fit())[1]

3.155200948041877e-40

### Question 10

Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

In [29]:
m4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data = botswana)
fitted = m4.fit(cov_type='HC1')
fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Tue, 08 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:24:34   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.152      0.000      -1.575      -0.565
age                0.1702      0.004     38.746      0.000       0.162       0.179
educ              -0.0729      0.007    -10.311      0.000      -0.087      -0.059
idlnchld           0.0770      0.014      5.323      0.000       0.049       0.105
knowmeth           0.5610      0.174      3.224      0.001       0.220       0.902
usemeth            0.6516      0.052     12.571      0.000       0.550       0.753
agefm             -0.0606      0.010     -6.192      0.000      -0.080      -0.041
heduc             -0.0573      0.009     -6.440      0.000      -0.075      -0.040
urban             -0.2190      0.045     -4.814      0.000      -0.308      -0.130
electric          -0.3207      0.063     -5.076      0.000      -0.445      -0.197
bicycle            0.2046      0.048      4.279      0.000       0.111       0.298
nevermarr         -2.2501      0.202    -11.158      0.000      -2.645      -1.855
idlnchld_noans     0.6565      0.216      3.043      0.002       0.234       1.079
heduc_noans       -0.8853      0.191     -4.638      0.000      -1.259      -0.511
usemeth_noans      0.7732      0.212      3.639      0.000       0.357       1.190
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""